In [1]:
import pandas as pd
from tqdm import tqdm_notebook

import sys
sys.path.append("../../../lib") # Adds higher directory to python modules path.
import helper_functions as hf
import language_processing as lp
import feature_extraction as fe

In [2]:
PATH = "../../../data/"
data_full = pd.read_csv(f'{PATH}train.tsv', sep='\t')
stopwords = [line.rstrip('\n') for line in open(f'{PATH}stopwords/english')]


## Replace NaN

In [3]:
data_full = hf.replace_nan(data_full)

In [4]:
data_full.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,No Brand,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard is in great condition and works ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top with a hint of lace and a key hol...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,No Brand,35.0,1,New with tags. Leather horses. Retail for [rm]...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,No Brand,44.0,0,Complete with certificate of authenticity


# Work with only 'Electronics/Video Games & Consoles/Video Gaming Merchandise' category

In [5]:
cat_df = data_full.loc[data_full.category_name == 'Electronics/Video Games & Consoles/Video Gaming Merchandise']

len(cat_df)

1151

## Delete items without description

In [6]:
cat_df = cat_df[cat_df.item_description != 'No description yet']

len(cat_df)

1065

## Extract labels

In [7]:
import price_classifier

pc = price_classifier.PriceClassifier(cat_df, 5)
y = pc.extract(cat_df)

for range_ in pc.ranges:
    print("range {} has {} items".format(range_, (y == (str(range_[0]) + "-" + str(range_[1]))).sum()))

range (0, 9.0) has 209 items
range (9.0, 15.0) has 193 items
range (15.0, 23.0) has 227 items
range (23.0, 38.0) has 218 items
range (38.0, 589.0) has 218 items


### Train & Test split

In [8]:
from sklearn.model_selection import train_test_split

df_train, df_test, y_train, y_test = train_test_split(cat_df, y, test_size=0.20, random_state=42)

### Convert GloVe into word2vec

In [9]:
# from gensim.scripts.glove2word2vec import glove2word2vec
# glove_input_file = f'{PATH}glove/glove.840B.300d.txt'
# word2vec_output_file = f'{PATH}glove/glove.840B.300d.txt.word2vec'
# glove2word2vec(glove_input_file, word2vec_output_file)

### Load the word2vec model

In [10]:
from gensim.models import KeyedVectors

# UNCOMMENT ONE OF THE FOLLOWING

# filename = f'{PATH}glove/glove.6B.100d.txt.word2vec' # GloVe Wikipedia + Gigaword
# model = KeyedVectors.load_word2vec_format(filename, binary=False)

filename = f'{PATH}glove/glove.6B.300d.txt.word2vec' # GloVe Wikipedia + Gigaword
model = KeyedVectors.load_word2vec_format(filename, binary=False)

# filename = f'{PATH}glove/glove.840B.300d.txt.word2vec' # GloVe Common Crowl
# model = KeyedVectors.load_word2vec_format(filename, binary=False)


# filename = f'{PATH}google/GoogleNews-vectors-negative300.bin'# Google News
# model = KeyedVectors.load_word2vec_format(filename, binary=True)

## Run grid search

Run grid search for all combinations of text processing

In [12]:
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

NLP_param_grid = {
    'desc_sw': [stopwords, None],
    'name_sw': [stopwords, None],
}

grid_search_param_grid = {
#     'n_estimators':[1, 3, 10, 30, 100, 300, 1000],
    'n_estimators':[1],
#     'max_depth':[1,3,10,30,100],
    'min_samples_leaf':[1,3,10],
    'random_state':[42]
}

optimisation_result = None

for gp in tqdm_notebook(list(ParameterGrid(NLP_param_grid))):
#     print("NLP params:", gp)

    # Create pipeline
    pipe = fe.Pipeline(steps=[
        ('item_desc', lp.MeanEmbeddingVectorizer(
            model,
            df_train,
            'item_description',
            stopwords=gp['desc_sw']
        )),
        ('name', lp.MeanEmbeddingVectorizer(
            model,
            df_train,
            'name',
            stopwords=gp['name_sw']
        )),
    ])
    
    # Exctract features
#     print("Extracting features")
    X_train = pipe.extract_features(df_train)
    X_test = pipe.extract_features(df_test)

    # Run grid search
#     print("Running grid search")
    random_forest = RandomForestClassifier()
    clf = GridSearchCV(random_forest, grid_search_param_grid, iid=False, cv=5, n_jobs=-1)
    clf.fit(X_train, y_train)
    
    current_result = {
        'best_score_': clf.best_score_,
        'best_NLP_gp': gp,
        'best_params_': clf.best_params_,
        'best_estimator_': clf.best_estimator_,
        'X_train': X_train,
        'X_test': X_test,
    }
    
    if optimisation_result is None:
        optimisation_result = current_result
#         print()
        continue
        
    if current_result['best_score_'] > optimisation_result['best_score_']:
        print("New best score achieved", current_result['best_score_'])
        optimisation_result = current_result
#     print()

# Add the parameters for grid search and the training data
print("Optimization done, adding additional data into optimisation_result before saving pickle...")
optimisation_result['NLP_param_grid'] = NLP_param_grid
optimisation_result['grid_search_param_grid'] = grid_search_param_grid
optimisation_result['df_train'] = df_train
optimisation_result['df_test'] = df_test
optimisation_result['y_train'] = y_train
optimisation_result['y_test'] = y_test

print("Saving pickle")
# hf.save_pickle(optimisation_result, f'{PATH}pickle/BOW_optimisation_result')
print("Done")

/Users/daniel/anaconda3/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/backend/semaphore_tracker.py:74: UserWarning: semaphore_tracker: process died unexpectedly, relaunching.  Some semaphores might leak.
  warnings.warn('semaphore_tracker: process died unexpectedly, '



Optimization done, adding additional data into optimisation_result before saving pickle...
Saving pickle
Done


### Exctract features

In [16]:
print("Extracting features for the training set...")
X_train = pipe.extract_features(df_train)
print("Done")
print("Extracting features for the test set...")
X_test = pipe.extract_features(df_test)
print("Done")

Extracting features for the training set...
Done
Extracting features for the test set...
Done


### Analysis on word embeddings (item_description)

In [17]:
voc_set_model = pipe.named_steps['word2vec_item_description'].voc_set_model
voc_set_df_train = pipe.named_steps['word2vec_item_description'].voc_set_df_train
voc_set_intersect = pipe.named_steps['word2vec_item_description'].voc_set_intersect

In [18]:
print("Size of model vocabulary: {}".format(len(voc_set_model)))
print("Size of df_train vocabulary: {}".format(len(voc_set_df_train)))
print("Size of intersect vocabulary: {}".format(len(voc_set_intersect)))

Size of model vocabulary: 400000
Size of df_train vocabulary: 3307
Size of intersect vocabulary: 2406


In [19]:
print("Intersect vocabulary: {}".format(voc_set_intersect))

Intersect vocabulary: {'us', 'kind', 'production', 'improves', 'control', 'applicant', 'outs', 'ab', 'smooth', 'literally', 'optically', 'reminder', 'aligning', 'webcam', 'alot', 'full', 'please', 'released', 'properly', 'slot', 'lighting', 'bangs', 'sticky', 'zte', 'enhanced', 'directly', 'lens', 'line', 'extension', 'fitted', 'll', 'high', 'specially', 'package', 'turn', 'very', 'panels', 'cup', 'yellowing', 'returned', 'burlap', '@', 'protection', 'your', 'rm', 'reinforces', 'ship', 'fit', 'maintains', 'typical', 'compliments', '4-6', 'said', 'prime', 'etched', 'cheapest', 'bucks', '360', 'point', 'packaged', 'nor', 'smartphones', 'separate', 'unbranded', 'one', '36', 'nwt', 'bluetooth', 'i', '8h', 'wrap', 'exposed', 'form', 'charger', 'worn', 'google', 'solve', 'diminish', 'trees', 'skin', 'military', '2.5', 'tough', 'sports', 'unscratched', 'fast', 's4', '1-3', 'holder', 'precise', 'allows', ';', 'proprietary', 'thinking', '20', 'ups', 'reduction', 'fyi', 'motorola', 'big', 'unloc

In [20]:
voc_diff = voc_set_df_train.difference(voc_set_model)
print("Vocabulary that is in df_train but is NOT in model:\nlentght: {}\nvoc: {}".format(len(voc_diff), voc_diff))

Vocabulary that is in df_train but is NOT in model:
lentght: 901
voc: {'screens*', 'item.our', '¶¶this', 'anit-shatter', 'techscreen', 'extemal', '2packs', 'case.4', 'discounts⬅️', 'bundle✨', '⚡️buy', 'free✨', '1xretail', '•high-response', 'hardness-3', '♦curved', 's7/s7edge', '-generic', 'anti-oil', '❤welcome', 'repair/replacing', 'shipping✅', '5se', '6+/s', 'plus..', 'nano-plated', '☆tempered', 'application/removal', 'willnot', 'amfilm', 'iphone7', 'guarenteed', '*opened', '24hrs***', '1.edge', 'anti-bubble', 'glass,9h', 'dust-free', 'anti-chip', 'oleophobic/hydrophobic', 'glass****', 'supershieldz', '↪message', '1×hard', 'note4/5/galaxy', 'plusthe', 'radiaetion', 'buying*', '****you', 'qty', 'watchonce', 'privary', '✳️see', '-zero-residue', 'tzumi', 'smudge/fingerprint-proof', '.message', 'iphone6plus', 'experience.optically', 'scratch-proof', '2.5d', 'lmirror', '▪️', '♦real', '*if', '1×', 'liquidskin', '⚡️all', '⭐️', '✨only', '⚡️fast', '2xtwo', 'save❤❤', 'iwatch', 'plus,6', '-price

### Random Forest

In [21]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100,
                             max_depth=100,
                             min_samples_leaf=1,
                             random_state=0
                            )
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=100, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [22]:
pred_train = clf.predict(X_train)
pred_test = clf.predict(X_test)

In [23]:
from sklearn.metrics import accuracy_score

acc_train = accuracy_score(y_train, pred_train)
print('Train accuracy: ', acc_train)

acc_test = accuracy_score(y_test, pred_test)
print('Test accuracy: ', acc_test)

Train accuracy:  0.9944237918215614
Test accuracy:  0.5613382899628253


# Naive algorithm: predicting average of the class

In [24]:
mode = y_train.mode()

pred_train_naive = y_train.copy()
pred_test_naive = y_test.copy()
pred_train_naive.loc[:] = mode.values[0]
pred_test_naive.loc[:] = mode.values[0]

In [25]:
print('Naive train accuracy: ', accuracy_score(y_train, pred_train_naive))
print('Naive test accuracy: ', accuracy_score(y_test, pred_test_naive))

Naive train accuracy:  0.29120198265179675
Naive test accuracy:  0.2732342007434944


In [26]:
# from importlib import reload
# fe = reload(fe)
# lp = reload(lp)